In [29]:
%pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import pandas as pd

**Especificar a continuación el nombre del archivo conteniendo el catálogo VRAI junto con las respuestas del formulario**

In [37]:
df_courses_vrai_file = 'formulario_limpio_rezagados.xlsx'

A continuación se procede a importar y formatear la información de los **cursos generales que los estudiantes indicaron** en el formulario

In [40]:
cols_df_students = ['Nombre completo', 'Transcript of Records', 'RUT UC', 'Correo electrónico2','País de tu universidad', 'Universidad de Origen', 'NRC', 'Sigla', 'Curso dentro del catálogo']
df_students = pd.read_excel(df_courses_vrai_file, sheet_name='Cursos validados', dtype=str)[cols_df_students].astype(str)
names_ruts = df_students[['Nombre completo', 'RUT UC']].drop_duplicates(subset=['RUT UC'])

A continuación se procede a importar y formatear la información de los cursos desde el **catálogo VRAI**

In [41]:
cols_df_courses = ['NRC', 'Escuela', 'Nombre Curso', 'Sigla', 'Sección', 'Campus', 'Horario Cátedra/Clase', 'Horario Ayudantía', 'Horario Laboratorio', 'Horario Taller']
df_courses = pd.read_excel(df_courses_vrai_file, sheet_name='Catálogo VRAI', dtype=str)[cols_df_courses].astype(str)

In [46]:
columns_successful_cases_df = ['Nombre estudiante', 'Transcript of Records', 'RUT UC', 'Correo electrónico', 'País de origen', 'Universidad de origen', 'Unidad académica', 'Nombre curso', 'NRC', 'Sigla', 'Número de sección', 'Prioridad curso', '¿Inscrito?', 'Observaciones', 'Asignación DMG']
columns_unsuccessful_cases_df = ['Nombre estudiante', 'RUT UC', 'Motivo']
successful_cases_df = pd.DataFrame(columns=columns_successful_cases_df)
unsuccessful_cases_df = pd.DataFrame(columns=columns_unsuccessful_cases_df)

def add_to_df_successful(df, student_info, courses, index_course1, index_course2):
    course1, course2 = courses[0], courses[1]
    student_name = student_info['name']
    tor = student_info['tor']
    rut_uc = student_info['rut_uc']
    email = student_info['email']
    uni_country = student_info['uni_country']
    uni_of_origin = student_info['uni_name']
    ua_1 = course1['Escuela'].values[0]
    course_name_1 = course1['Nombre Curso'].values[0]
    nrc_1 = course1['NRC'].values[0]
    sigla_1 = course1['Sigla'].values[0]
    section_number_1 = course1['Sección'].values[0]
    ua_2 = course2['Escuela'].values[0]
    course_name_2 = course2['Nombre Curso'].values[0]
    nrc_2 = course2['NRC'].values[0]
    sigla_2 = course2['Sigla'].values[0]
    section_number_2 = course2['Sección'].values[0]
    df.loc[len(df)] = [student_name, tor, rut_uc, email, uni_country, uni_of_origin, ua_1, 
                       course_name_1, nrc_1, sigla_1, section_number_1, index_course1,'', '', '']
    df.loc[len(df)] = [student_name, tor, rut_uc, email, uni_country, uni_of_origin, ua_2, 
                       course_name_2, nrc_2, sigla_2, section_number_2, index_course2,'', '', '']
    
def add_to_df_unsuccessful(df, student_info, reason):
    df.loc[len(df)] = [student_info['name'], student_info['rut_uc'], reason]

In [47]:
priority_values = {1 : 150, 2 : 130, 3 : 110, 4 : 100, 5 : 70, 6 : 60, 7 : 50, 8 : 40, 9 : 30, 10 : 20}
start_times_info = { '0820' : 1, '0940' : 2, '1100' : 3, '1220' : 4, '1450' : 5, '1610' : 6, '1730' : 7, '1850' : 8, '2010' : 9}
end_times_info = { '0930' : 1, '1050' : 2, '1210' : 3, '1330' : 4, '1600' : 5, '1720' : 6, '1840' : 7, '2000' : 8, '2120' : 9}

def format_schedule_info(info):
    return info.strip().replace(' ', '').replace(':', '').replace('\n', '')[:-1].split(';')

def generate_priority_combinations(n_courses):
    all_combinations = []
    for i in range(1, n_courses + 1):
        for j in range(i + 1, n_courses + 1):
            all_combinations.append([[i, j], i + j])
    all_combinations.sort(key = lambda x : x[1])
    return all_combinations

def transform_module_info(schedule1:dict, full_schedule_info:list):
    for info in full_schedule_info:
        info_list = format_schedule_info(info)
        for element in info_list:
            week_day, start_time, end_time = element[-10], element[-9:-5], element[-4:]
            start_module, end_module = start_times_info[start_time], end_times_info[end_time]
            for i in range(start_module, end_module + 1):
                schedule1[week_day].append(i)
    return schedule1

def get_full_schedule(course_match):
    campus = course_match['Campus'].values[0].strip()
    catedra_schedule, ayudantia_schedule = course_match['Horario Cátedra/Clase'].values[0], course_match['Horario Ayudantía'].values[0]
    taller_schedule, lab_schedule = course_match['Horario Taller'].values[0], course_match['Horario Laboratorio'].values[0]
    full_schedule_list = [x for x in [catedra_schedule, ayudantia_schedule, taller_schedule, lab_schedule] if x != 'nan']
    return full_schedule_list, campus

def get_class1_info(course_match): # Extraer la información de los horarios del curso 1, convertirla y transferirla al diccionario schedule1.
    schedule1 = {'L': [], 'M' : [], 'W' : [], 'J' : [], 'V' : [], 'S' : []}
    full_schedule_info, campus1 = get_full_schedule(course_match)
    schedule1 = transform_module_info(schedule1, full_schedule_info)
    return schedule1, campus1

def check_class2_compatibility(schedule1, campus1, course_match):
    full_schedule_info, campus2 = get_full_schedule(course_match)
    for info in full_schedule_info:
        info_list = format_schedule_info(info)
        for element in info_list:
            week_day, start_time, end_time = element[-10], element[-9:-5], element[-4:]
            start_module, end_module = start_times_info[start_time], end_times_info[end_time]
            for i in range(start_module, end_module + 1):
                if i in schedule1[week_day] or (campus1 != campus2 and (i + 1 in schedule1[week_day] or i - 1 in schedule1[week_day])):
                    return False
    return True

def check_compatibility(nrc1, nrc2, sigla1, sigla2):
    course1_match = df_courses.loc[df_courses['NRC'] == nrc1]
    course2_match = df_courses.loc[df_courses['NRC'] == nrc2]
    if len(course1_match) and len(course2_match) and sigla1 != sigla2:
        schedule1, campus1 = get_class1_info(course1_match)
        compatible = check_class2_compatibility(schedule1, campus1, course2_match)
        return compatible, [course1_match, course2_match]
    return False, []

In [48]:
# ['Nombre completo', 'Transcript of Records', 'RUT UC', 'Correo electrónico2','País de tu universidad', 'Universidad de Origen', 'NRC', 'Sigla', 'Curso dentro del catálogo']
for index, row in names_ruts.iterrows():
    compatible = False
    matches = df_students.loc[(df_students['RUT UC'] == row['RUT UC']) & (df_students['Curso dentro del catálogo'] == 'Sí')]
    if len(matches) >= 2:
        student_info = { 'name': matches['Nombre completo'].values[0], 'tor': matches['Transcript of Records'].values[0], 
                     'rut_uc': matches['RUT UC'].values[0], 'email': matches['Correo electrónico2'].values[0], 
                     'uni_country': matches['País de tu universidad'].values[0], 
                     'uni_name' : matches['Universidad de Origen'].values[0] }
        all_priority_combinations = generate_priority_combinations(len(matches))
        for priority_combination in all_priority_combinations:
            index_course1, index_course2 = priority_combination[0][0], priority_combination[0][1] 
            nrc_course1, nrc_course2 = matches['NRC'].values[index_course1 - 1], matches['NRC'].values[index_course2 - 1]
            sigla_course1, sigla_course2 = matches['Sigla'].values[index_course1 - 1], matches['Sigla'].values[index_course2 - 1]
            compatible, courses = check_compatibility(nrc_course1, nrc_course2, sigla_course1, sigla_course2)
            if compatible:
                add_to_df_successful(successful_cases_df, student_info, courses, index_course1, index_course2)
                break

        if not compatible:
            reason = 'No se encontraron dos cursos compatibles.'
            add_to_df_unsuccessful(unsuccessful_cases_df, student_info, reason)
    else:
        reason = 'Estudiante tiene menos de dos opciones de curso.'
        student_info = { 'name': row['Nombre completo'], 'rut_uc' : row['RUT UC']}
        add_to_df_unsuccessful(unsuccessful_cases_df, student_info, reason)

with pd.ExcelWriter('Revisión tope de horarios cursos e inscripción_rezagados.xlsx') as writer:
    successful_cases_df.to_excel(writer, sheet_name='Con éxito', index=False)
    unsuccessful_cases_df.to_excel(writer, sheet_name='Sin éxito', index=False)